In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!apt-get install -y python-numpy python-dev cmake zlib1g-dev libjpeg-dev xvfb libav-tools xorg-dev python-opengl libboost-all-dev libsdl2-dev swig

In [0]:
!pip install gym
!pip install keras

#### Kütüphanelerin İmport Edilmesi

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D, Dense
from keras.layers import Dropout, Activation , Flatten
from google.colab import files
import numpy as np
import matplotlib.pyplot as plt
import random

%matplotlib inline

#### Kod Başlangıcı

In [0]:

NUM_CLASSES=10   # 10 Kategori olduğu için.

(X_train, y_train), (X_test, y_test) = cifar10.load_data()# Datayı yüklüyoruz.İndiriyoruz.

In [0]:
TYPE_MAP = {
    0: u'Uçak',
    1: 'Otomobil',
    2: u'Kuş',
    3: 'Kedi',
    4: 'Geyik',
    5: u'Köpek',
    6: u'Kurbağa',
    7: 'At',
    8: 'Gemi',
    9: 'Kamyon'
} # Verilerin içerisinde ki isimlendirmeler. https://www.kaggle.com/c/cifar-10/data

In [0]:
X_train = X_train.astype('float32') / 225.0   # Veri tiplerini sıfır ile bir arasına çekmemize yarıyor.
X_test = X_test.astype('float32') / 225.0

y_train = keras.utils.to_categorical(y_train,NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test,NUM_CLASSES)

In [0]:
dataset_size = X_train.shape[0]

idx = random.randint(0, dataset_size)

sample_img = X_train[idx]
sample_label= np.argmax(y_train[idx])

print(u"Resmin Cinsi : %s" % TYPE_MAP[sample_label]) # Rasgele çektiğimiz resmin ne olduğu ile birlikte aşağıda göstertiyoruz. Daha eğitim yapmadık.Sadece dataya bakiyoruz.
plt.imshow(sample_img)

# New Section

In [0]:
input_shape = X_train.shape[1:]

model = Sequential()

model.add(Conv2D(32,(5,5),padding ='same',input_shape=input_shape)) # 5x5x32'lik bir Conv uyguluyoruz.
model.add(Activation('relu')) # Activasyon fonksiyonu relu. 

model.add(Conv2D(32, (3, 3)))# 3x3x32'lik bir Conv yapıyoruz.
model.add(MaxPool2D(pool_size=(2,2))) # 2x2 'lik bir  max pool yaparak veriyi daha aza indirgiyoruz.
model.add(Activation('relu'))
model.add(Dropout(0.25)) #

model.add(Conv2D(64,(3,3),padding='same')) # 3x3x64'lük bir Conv işlemi. padding var.
model.add(Activation('relu')) # Activatiyonu relu

model.add(Conv2D(64,(3,3))) # 3x3x64'lük bir conv işlemi paddingsiz
model.add(MaxPool2D(pool_size=(2,2)))# 2x2'lik maxpool
model.add(Dropout(0.5)) # Dropout rasgele olarak verileri sıfıra çekiyor.

model.add(Flatten()) # Bu katman tek dikey bir dizi şeklini aldırıyor çikişlarımızın
model.add(Dense(1024,activation='relu')) # Hidden Layer
model.add(Dropout(0.25))


model.add(Dense(NUM_CLASSES,activation='softmax'))  # Softmax ile çikişi gösteriyoruz.

model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy']) # Optimizer ile hatayi daha aza indirgeme işlemi yapıyoruz. Adadelta kullandığımız algortima Adam vb de kullanılabilir.

In [0]:
model.fit(X_train, y_train, batch_size=512, epochs=10,verbose=1,validation_data=(X_test, y_test),shuffle=True)  # Batch_size Her iterasyonda kaç resim göndereceğimiz.Fazla resim göndermek doğrulu arttırsada ezberide arttırıyor.İdeal 128 256 gibi. Epoch iterasyon kaç kere yapılcak bu.Validation= doğruluk teste göre oranlicak

In [0]:
model.evaluate(X_test,y_test, verbose=1) # Modelin değerlendirmesini yapıyor.Valitadon???.

In [0]:
idx = random.randint(0, X_test.shape[0])# Rasgele resim seçiyoruz

sample_img= X_test[idx]
sample_label = np.argmax(y_test[idx])

print(u"Resmin Cinsi : %s" % TYPE_MAP[sample_label] ) # Rasgele resimin ne olduğunu veya söylediğini gösteriyoruz
plt.imshow(sample_img)

sample_img= sample_img.reshape(1, 32, 32 ,3) # 32 32 Boyutunda olduğu için ve RGB olduğundan 3 diyoruz.

for i, v in enumerate(model.predict(sample_img)[0]): # Resimin kaç ihtimalle olduğunu yazdiriyoruz.
  print(u"Resim %s olma ihtimali : %.2f%%" % (TYPE_MAP[i],v *100)) # İhtimallini yansıtıyoruz i değişkeni diğer kategorilere göre de ihtimalleri göstermemize yariyor.

In [0]:
model.save('cifar10-cnn.h5') # Eğitilmiş verileri kaydetmek için Drive'a direkt kaydediyor.

In [0]:
model.load_weights('cifar10-cnn.h5') # Eğitilmiş ağırlıkları yüklemek için.

In [0]:
!ls

In [0]:
model.summary() # modelimizin akış seması diyebiliriz diyagramı ve parametre sayıları.